Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 10 µs, sys: 4.01 ms, total: 4.02 ms
Wall time: 9.36 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.60/10336/1 Dashboard: http://10.20.0.60:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 232 ms, sys: 22.9 ms, total: 255 ms
Wall time: 2.02 s


[1.9534416464478983,
 1.1344832185505365,
 1.5780868466149336,
 1.637993844713782,
 1.1310320949391972,
 1.9235003724711008,
 1.2129925202728034,
 1.3744149197368203,
 1.9022233087366858,
 1.5971415600209218,
 1.6272387462623088,
 1.4012689477236986,
 1.324202586101242,
 1.1952207721733084,
 1.5154394209494726,
 1.0414682295602817,
 1.950099911477449,
 1.5594813281907431,
 1.9884080104610677,
 1.5269841052370583,
 1.247712686424796,
 1.2088604728706174,
 1.431376774588208,
 1.7100478105709669,
 1.2159964888314596,
 1.3375101327419658,
 1.3404390595396674,
 1.7389681074479195,
 1.6157181876176119,
 1.24418532747218,
 1.6638385340043076,
 1.2384587798721212,
 1.3496340651519625,
 1.7952711972723776,
 1.9771684030536,
 1.4266536378442116,
 1.3541621190921749,
 1.5388916882985102,
 1.4741475661819918,
 1.51268351436436,
 1.956531582912606,
 1.0763768403094538,
 1.416584063450789,
 1.735777388071889,
 1.8342719427031553,
 1.3513040018856382,
 1.0162754682533097,
 1.6162170682563102,
 1.1216

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.7591018616499268,
 1.423150793309358,
 1.1269453945701817,
 1.8228750467912516,
 1.694452285444807,
 1.3774982316195377,
 1.594911831496297,
 1.2362885828398777,
 1.9100273835877326,
 1.476662720439104,
 1.7201660437101478,
 1.6797279194311372,
 1.6020548435798228,
 1.856265833438853,
 1.1175714017341898,
 1.1230299805750743,
 1.0788630967276784,
 1.5390371712275512,
 1.2958119730753732,
 1.80318884683809,
 1.0502591821621543,
 1.2001693801706528,
 1.35723208431717,
 1.2379428664369967,
 1.6123816030196712,
 1.2632611948688885,
 1.2526481735873696,
 1.6155737739623046,
 1.936035259253733,
 1.602204164657427,
 1.3979463966122387,
 1.6288729987019526,
 1.3736876530767996,
 1.4416977249812066,
 1.4167840596877284,
 1.3037594873824063,
 1.948239769473439,
 1.3814132110460853,
 1.8933734658162513,
 1.5609980405769315,
 1.4786703321781052,
 1.4220756034535955,
 1.9620197498894396,
 1.3890417794998537,
 1.5375294978262846,
 1.3817800992655704,
 1.810995644038777,
 1.7115982889396903,
 1.21

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 161 ms, sys: 16.2 ms, total: 177 ms
Wall time: 2.26 s
